In [ ]:
import gradio as gr
import time
import random
import os
import json
from dotenv import load_dotenv
import requests

# Add OpenAI import
from openai import AzureOpenAI

In [ ]:
endpoint = ""
api_key = ""
search_key = ""
search_endpoint=""
index_name = ""
semantic_configuration = ""

In [ ]:
# DALL-E-3
end_point_2 = ""
api_key_2 = ""

In [ ]:
import gradio as gr
import requests

def get_message_history(chat_history):
    history_list = list()
    history_length = 3

    for chat in chat_history[-history_length:]:
        #print(chat[0], chat[1])
        history_list.append({"role": "user","content": chat[0]})
        history_list.append({"role": "assistant","content": chat[1]})
    return history_list

def request_gpt(prompt, history_list):

    # 1. endpoint

    # 2. header
    headers = {
        "Content-Type":"application/json",
        "api-key": api_key
    }
    
    
    # 이전 메세지와 입련된 prompt 처리하기 
    message_list = list()
    message_list.append({"role": "system","content":"당신이 요청한 내용을 더 명확하고 실행 가능한 전문가 시스템으로 구성하기 위해 지침을 재정리하겠습니다.\n\n---\n\n당신은 인체기능해부학의 전문가로, 정밀한 의학 및 생리학적 지식을 바탕으로 질문에 대한 답변을 제공하는 것이 목표입니다. 질문자가 제시한 주제에 대해 구체적이고 정확한 정보를 전달하며, 필요시 예시 및 시각 자료 설명을 포함합니다.\n\n# 지침\n\n1. **전문적이고 명확한 표현**: 전문적인 어휘와 설명을 사용하되, 질문자의 수준에 맞춰 쉽게 이해할 수 있는 단어와 문장을 선택합니다.\n2. **정확한 정보 제공**: 최신 의학 표준 및 연구에 근거한 정보를 제공합니다.\n3. **단계적 설명**: 복잡한 개념은 단계적으로 분리하여 설명합니다.\n4. **사용자의 추가 요청 반영**: 사용자가 추가 정보를 요청할 경우, 그것을 구체적으로 반영합니다.\n5. **필요시 그림 및 도표의 언급**: \"이 부위는 신체의 특정 위치에 있으니, 일반적인 인체 해부학 도표의 [위치]를 참조하세요.\" 같은 방식으로 시각적 자료를 지원합니다.\n6. **가능한 출력 형식**: 텍스트 (및 JSON 형식)이 요청될 경우 조직화된 형태로 제공할 수 있습니다.\n\n# 출력 포맷\n\n- 간략한 문단형 설명 또는 단계별 개요.\n- 특정 데이터를 포함한다면 표 형식 또는 JSON으로 간결하게 요약 가능.\n\n# 예시\n\n**질문**: \"팔꿈치 관절의 주요 기능과 구조를 설명해주세요.\"\n\n**답변**:\n\n팔꿈치(Elbow joint)는 신체에서 중요한 힌지 관절 중 하나로서, 주로 아래의 세 가지 주요 구조로 구성되어 있습니다:\n\n1. **골격 구조**:\n   - 상완골(Humerus), 요골(Radius) 및 척골(Ulna)이 팔꿈치를 형성합니다.\n2. **관절과 인대**:\n   - 주관절(Main Joint): Humeroulnar joint (주로 굴곡과 신전 운동을 담당)\n   - 일부 안정성을 제공하는 인대: 내측 측부인대(Ulnar collateral ligament)와 외측 측부인대(Radial collateral ligament)\n3. **주요 근육**:\n   - 상완이두근(Biceps Brachii): 팔꿈치 굴곡에 기여\n   - 상완삼두근(Triceps Brachii): 팔꿈치 신전에 기여\n\n### 주요 기능:\n- 완전한 팔의 이동성 (Flexion and Extension)\n- 안정적인 아래팔의 위치 조절 (Pronation/Supination을 돕는 관절 포함)\n\n### 도움말:\n자세한 해부학적 위치는 표준 해부학 도표에서 \"상완-요골-척골 연결 구조\"를 참조하면 시각적 요소가 더 이해를 도울 것입니다.\n\n\n---\n### 참조\n데이터에 있는내용을 적극적으로 활용할것!\n\n---\n필요한 경우 구체적인 요청사항을 자유롭게 알려주세요!"})
    # fewshot은 여기에....
    message_list.extend(history_list)
    message_list.append({"role": "user","content":prompt})

    # 3. body
    paylod = {
        "messages": message_list,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800,
        "data_sources": [{
          "type": "azure_search",
          "parameters": {
            "endpoint": f"{search_endpoint}",
            "index_name": index_name,
            "semantic_configuration": semantic_configuration,
            "query_type": "semantic",
            "fields_mapping": {},
            "in_scope": True,
            "role_information": "당신이 요청한 내용을 더 명확하고 실행 가능한 전문가 시스템으로 구성하기 위해 지침을 재정리하겠습니다.\n\n---\n\n당신은 인체기능해부학의 전문가로, 정밀한 의학 및 생리학적 지식을 바탕으로 질문에 대한 답변을 제공하는 것이 목표입니다. 질문자가 제시한 주제에 대해 구체적이고 정확한 정보를 전달하며, 필요시 예시 및 시각 자료 설명을 포함합니다.\n\n# 지침\n\n1. **전문적이고 명확한 표현**: 전문적인 어휘와 설명을 사용하되, 질문자의 수준에 맞춰 쉽게 이해할 수 있는 단어와 문장을 선택합니다.\n2. **정확한 정보 제공**: 최신 의학 표준 및 연구에 근거한 정보를 제공합니다.\n3. **단계적 설명**: 복잡한 개념은 단계적으로 분리하여 설명합니다.\n4. **사용자의 추가 요청 반영**: 사용자가 추가 정보를 요청할 경우, 그것을 구체적으로 반영합니다.\n5. **필요시 그림 및 도표의 언급**: \"이 부위는 신체의 특정 위치에 있으니, 일반적인 인체 해부학 도표의 [위치]를 참조하세요.\" 같은 방식으로 시각적 자료를 지원합니다.\n6. **가능한 출력 형식**: 텍스트 (및 JSON 형식)이 요청될 경우 조직화된 형태로 제공할 수 있습니다.\n\n# 출력 포맷\n\n- 간략한 문단형 설명 또는 단계별 개요.\n- 특정 데이터를 포함한다면 표 형식 또는 JSON으로 간결하게 요약 가능.\n\n# 예시\n\n**질문**: \"팔꿈치 관절의 주요 기능과 구조를 설명해주세요.\"\n\n**답변**:\n\n팔꿈치(Elbow joint)는 신체에서 중요한 힌지 관절 중 하나로서, 주로 아래의 세 가지 주요 구조로 구성되어 있습니다:\n\n1. **골격 구조**:\n   - 상완골(Humerus), 요골(Radius) 및 척골(Ulna)이 팔꿈치를 형성합니다.\n2. **관절과 인대**:\n   - 주관절(Main Joint): Humeroulnar joint (주로 굴곡과 신전 운동을 담당)\n   - 일부 안정성을 제공하는 인대: 내측 측부인대(Ulnar collateral ligament)와 외측 측부인대(Radial collateral ligament)\n3. **주요 근육**:\n   - 상완이두근(Biceps Brachii): 팔꿈치 굴곡에 기여\n   - 상완삼두근(Triceps Brachii): 팔꿈치 신전에 기여\n\n### 주요 기능:\n- 완전한 팔의 이동성 (Flexion and Extension)\n- 안정적인 아래팔의 위치 조절 (Pronation/Supination을 돕는 관절 포함)\n\n### 도움말:\n자세한 해부학적 위치는 표준 해부학 도표에서 \"상완-요골-척골 연결 구조\"를 참조하면 시각적 요소가 더 이해를 도울 것입니다.\n\n\n---\n### 참조\n데이터에 있는내용을 적극적으로 활용할것!\n\n---\n필요한 경우 구체적인 요청사항을 자유롭게 알려주세요!",
            "filter": None,
            "strictness": 3,
            "top_n_documents": 5,
            "authentication": {
              "type": "api_key",
              "key": f"{search_key}"
            }
          }
        }]
    }
    # 4. method - post
    response = requests.post(endpoint, headers=headers, json=paylod)

    if response.status_code == 200:
        response_json = response.json()
        content_text = response_json["choices"][0]['message']['content']
        if response_json["choices"][0]['message']['context']["citations"]:
            display(response_json["choices"][0]['message']['context']["citations"])
            citiation = []
            for i in response_json["choices"][0]['message']['context']["citations"]:
                citiation.append(i["content"])
        else:
            citiation = ""
            
        return content_text, citiation
    else:
        return f'{response.status_code}, {response.text}', ""
    
def make_img(input_text):
    
    #header
    header = {
        'Content-Type':'application',
        'api-key':api_key_2
    }
    #body
    body = {
        "prompt": input_text,
        "n": 1,
        "size": "1024x1024"
    }

    response = requests.post(url=end_point_2, headers=header, json=body)
    url = response.json()["data"][0]["url"]
    
    if response.status_code == 200:
        return url
    else:
        return ""

    
def click_send(input_message, chat_history ):

    history_list = get_message_history(chat_history)
    bot_message, citiation = request_gpt(input_message, history_list)
    chat_history.append((input_message, bot_message))
    return "", chat_history, citiation

with gr.Blocks() as demo:

    #chatbot = gr.Chatbot(type="messages")
    chatbot = gr.Chatbot()
    citiation = gr.HTML(label="참조")
    input_textbox = gr.Textbox(label="" , scale=7)
    submit_button = gr.Button("전송")
    img_box = gr.Image()
    input_imgbox = gr.Textbox(label="만들고 싶은 이미지 프롬프트를 입력하세요")
    img_button = gr.Button("전송")

    submit_button.click(fn=click_send, inputs=[input_textbox, chatbot], outputs=[input_textbox, chatbot, citiation])
    input_textbox.submit(fn=click_send, 
                         inputs=[input_textbox, chatbot], 
                         outputs=[input_textbox, chatbot, citiation])
    img_button.click(fn=make_img, inputs=input_imgbox, outputs=img_box)

demo.launch() #share=True

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
